In [2]:
#HAAAAARD

from typing import Union, List
class RingElement:
#Базовый класс для элементов колец: целых чисел и полиномов
    def __init__(self, data: Union[int, List[float]]):
#data:
#- если это целое число → элемент кольца Z;
#- если это список коэффициентов [a0, a1, ..., an] → полином a0 + a1*x + ... +
#,→ an*x^n.
        self.data = data # сами коэффы
        self.is_polynomial = isinstance(data, list) # bool кольцо Z или кольцо полиномов
    def __repr__(self) -> str: # красиво выводит элемент кольца
        if self.is_polynomial:
            terms = [f"{c}*x^{i}" if i > 0 else str(c) for i, c in enumerate(self.data) if c != 0]
            return " + ".join(terms) if terms else "0"
        return str(self.data)
    
def gcd_euclidean(a: int, b: int) -> int: # алгоритм евклида для целых чисел
    a, b = abs(a), abs(b)
    if a < b:
        a, b = b, a
    #цикл алгоритма Евклида
    while b != 0:
        a, b = b, a % b
    return a

def gcd_euclidean_list(numbers: List[int]) -> int: # алгоритм евклида для списка целых чисел
    result = abs(numbers[0])
    for num in numbers[1:]:
        result = gcd_euclidean(result, abs(num)) # НОД(a, b, c) = НОД(c, НОД(a, b))
    return result

def normalize_polynomial(polynomial: List[float]) -> List[float]:  # [1, 2, 3, 0, 0, 0] → [1, 2, 3]
    if not polynomial:
        return [0.0]
    i = len(polynomial) - 1 # старший коэффициент
    while i >= 0 and abs(polynomial[i]) <= 10**(-9): # flоat неточности
        i -= 1
    if i >= 0:
        return polynomial[:i+1] # срез без нулей 
    else:
        return [0.0]
    
def polynomial_is_zero(polynomial: List[float]) -> bool:
    if not polynomial:
        return True
    else:
        for coeff in polynomial:
            if abs(coeff) > 10**(-9):
                return False
    return True

def polynomial_division(a: List[float], b: List[float]) -> tuple: # деление полиномов
    a = normalize_polynomial(a)
    b = normalize_polynomial(b)
    if polynomial_is_zero(b):
        raise ValueError("деление на нулевой полином")
    if len(b) > len(a): # степень делимого меньше степени делителя
        return [0.0], a
    remainder = a.copy() # чтобы не менялся исходный список
    quotient = [0.0] * (len(a) - len(b) + 1) # массив для хранение частного(целой части)
    while len(remainder) >= len(b) and not polynomial_is_zero(remainder):
        current_coeff = remainder[-1] / b[-1]
        shift = len(remainder) - len(b) # на какую степень нужно умножать делитель
        quotient[shift] = current_coeff # записали в остаток
        for i in range(len(b)): # вычитаем для дальнейшего деления 
            remainder[i + shift] -= current_coeff * b[i] # [4, 3, 2, 1] -> [4, 3, (2-1), (1-1)]
        remainder = normalize_polynomial(remainder)
    return quotient, remainder
    
def gcd_polynomials(a: List[float], b: List[float]) -> List[float]: # НОД для полиномов
    a = normalize_polynomial(a)
    b = normalize_polynomial(b)
    if len(a) < len(b):
        a, b = b, a
    while not polynomial_is_zero(b):
        quotient, remainder = polynomial_division(a, b)
        a, b = b, normalize_polynomial(remainder)
    return normalize_polynomial(a)

def gcd_polynomials_list(polynomials: List[float]) -> List[float]:
    if not polynomials:
        return [0]
    result = polynomials[0]
    for polynom in polynomials[1:]:
        result = gcd_polynomials(result, polynom)
    return result

def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
#возвращает порождающий главного идеала, порождённого заданными элементами.
#- для Z: НОД целых чисел.
#- для K[x]: НОД полиномов (с коэффициентами в поле K).
    Type = elements[0].is_polynomial
    for elem in elements:
        if elem.is_polynomial != Type:
            raise TypeError("все элементы должны быть одного типа")
    # целые числа 
    if not Type:
        numbers = []
        for elem in elements:
            numbers.append(elem.data)
        return RingElement(gcd_euclidean_list(numbers))
    else:
        polynomials = []
        for elem in elements:
            polynomials.append(elem.data)
        return RingElement(gcd_polynomials_list(polynomials))
    
def analyze_ideal(elements: List[RingElement]) -> dict:
    if not elements:
        return {
            "главный?": True,
            "порождающий элемент": RingElement(0),
            "тип кольца": "не определен",
            "элементы": elements
        }
    
    # проверяем согласованность типов
    ring_type = 'полиномы' if elements[0].is_polynomial else 'целые числа'
    for elem in elements:
        if elem.is_polynomial != elements[0].is_polynomial:
            raise TypeError("Все элементы должны быть из одного кольца")
    
    # вычисляем порождающий через НОД
    generator = gcd_ring_elements(elements)
    
    return {
        "главный?": True,  # все идеалы главные
        "порождающий элемент": generator,
        "тип кольца": ring_type,
        "элементы": elements,
        "обоснование": f"{'K[x]' if elements[0].is_polynomial else 'Z'} - кольцо главных идеалов"
    }

def belongs_to_ideal(element: RingElement, ideal_generator: RingElement) -> bool: # принадлежность элемента идеала
    if element.is_polynomial != ideal_generator.is_polynomial:
        raise TypeError("Элементы должны быть из одного кольца")
    
    if not element.is_polynomial:
        # для Z: element ∈ (d) ⇔ d делит element
        if ideal_generator.data == 0:
            return element.data == 0
        return element.data % ideal_generator.data == 0
    else:
        # для K[x]: f ∈ (g) ⇔ g делит f
        if polynomial_is_zero(ideal_generator.data):
            return polynomial_is_zero(element.data)
        
        _, remainder = polynomial_division(element.data, ideal_generator.data)
        return polynomial_is_zero(remainder)
    
def run_simple_tests():
    print("ТЕСТЫ")
    
    # простые числа
    test1 = [RingElement(12), RingElement(18)]
    result1 = analyze_ideal(test1)
    print(f"НОД(12, 18) = {result1['порождающий элемент']}")
    
    # содержит ли идеал (6) число 24?
    belongs1 = belongs_to_ideal(RingElement(24), RingElement(6))
    print(f"24 ∈ (6)? {belongs1}")
    
    # содержит ли идеал (6) число 7?
    belongs2 = belongs_to_ideal(RingElement(7), RingElement(6))
    print(f"7 ∈ (6)? {belongs2}")
    
    # простые полиномы
    poly1 = RingElement([1, 0, 1])
    poly2 = RingElement([1, 1])     
    test2 = [poly1, poly2]
    result2 = analyze_ideal(test2)
    print(f"НОД(x^2+1, x+1) = {result2['порождающий элемент']}")
    
    # принадлежность полинома идеалу
    test_poly = RingElement([2, 2, 2, 2])  
    belongs3 = belongs_to_ideal(test_poly, result2['порождающий элемент'])
    print(f"2x^3+2x^2+2x+2 ∈ идеалу? {belongs3}")
    
    # деление полиномов
    a = [1, 0, 0, -1] 
    b = [1, -1]       
    quotient, remainder = polynomial_division(a, b)
    print(f"(x^3-1)/(x-1) = частное: {quotient}, остаток: {remainder}")
    
    # нулевой идеал
    test_zero = [RingElement(0)]
    result_zero = analyze_ideal(test_zero)
    print(f"Идеал (0): {result_zero['порождающий элемент']}")
    
    # один элемент
    test_single = [RingElement(15)]
    result_single = analyze_ideal(test_single)
    print(f"Идеал (15): {result_single['порождающий элемент']}")

run_simple_tests()

ТЕСТЫ
НОД(12, 18) = 6
24 ∈ (6)? True
7 ∈ (6)? False
НОД(x^2+1, x+1) = 2.0
2x^3+2x^2+2x+2 ∈ идеалу? True
(x^3-1)/(x-1) = частное: [1.0, 1.0, 1.0], остаток: [0.0]
Идеал (0): 0
Идеал (15): 15
